In [1]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
seed = 5
numpy.random.seed(seed)

In [3]:
# load dataset
dataframe = pandas.read_csv("mydata1.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:3]
Y = dataset[:,3]

In [4]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [5]:
# define baseline model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=3, activation='relu'))
	model.add(Dense(2, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [6]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

In [7]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [8]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 88.33% (13.02%)


In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, dummy_y, test_size=0.33, random_state=seed)
estimator.fit(X_train, Y_train)
predictions = estimator.predict(X_test)
print(predictions)
print(encoder.inverse_transform(predictions))
print(X_test)

[1 1 1 1 1 1 0 0 0 0 1 1 0 1 0 1 0 1 1 1]
['yes' 'yes' 'yes' 'yes' 'yes' 'yes' 'no' 'no' 'no' 'no' 'yes' 'yes' 'no'
 'yes' 'no' 'yes' 'no' 'yes' 'yes' 'yes']
[[5 5 5]
 [7 8 1]
 [5 8 5]
 [4 5 4]
 [6 5 9]
 [1 5 1]
 [1 4 5]
 [1 1 5]
 [4 1 4]
 [4 2 2]
 [4 4 1]
 [6 5 4]
 [2 1 1]
 [1 6 1]
 [1 3 1]
 [6 5 9]
 [1 1 3]
 [2 4 2]
 [7 5 4]
 [1 5 1]]
